In [1]:
# Standard library imports
import os

# Third party library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
BLINDED_FILE_PATH = "../datasets/TASK_2/TASK_2/blinded_test_set.csv"
blinded_test_data = pd.read_csv(BLINDED_FILE_PATH)

In [3]:
# Reading the csv file

FILE_PATH = "../datasets/TASK_2/TASK_2/train_set.csv"
train_data = pd.read_csv(FILE_PATH)

In [4]:
TEST_PATH = "../datasets/TASK_2/TASK_2/test_set.csv"
test_data = pd.read_csv(TEST_PATH)

In [5]:
train_data['CLASS'].value_counts().reset_index()

,CLASS,count
0,0,191
1,1,124


In [6]:
test_data['CLASS'].value_counts().reset_index()

,CLASS,count
0,0,58
1,1,42


In [7]:
X_test = test_data.drop(columns=["ID", "CLASS"])
y_test = test_data["CLASS"]

In [8]:
X_train = train_data.drop(columns=["ID", "CLASS"])
y_train = train_data["CLASS"]

In [9]:
y_train.isna().sum()

np.int64(0)

In [10]:
missing_per_row = X_train.isnull().sum(axis=1)
missing_per_row.value_counts().reset_index()


,index,count
0,0,199
1,23,116


In [11]:
import numpy as np

# Count infinities
np.isinf(X_train.to_numpy()).sum()


np.int64(4)

In [12]:
inf_mask = np.isinf(X_train)
inf_summary = inf_mask.sum(axis=0)
inf_summary[inf_summary > 0]

Feature_72    2
Feature_90    2
dtype: int64

In [13]:
X_clean = X_train.replace([np.inf, -np.inf], np.nan)

In [14]:
X_test = X_test.replace([np.inf, -np.inf], np.nan)

In [15]:
X_train.iloc[50].isnull().value_counts().reset_index()

,50,count
0,False,3215
1,True,23


In [16]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
X_imputed = imputer.fit_transform(X_clean)
X_test = imputer.transform(X_test)

In [17]:
from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()
X_scaled = scalar.fit_transform(X_imputed)
X_test = scalar.transform(X_test)

In [18]:
scalar.mean_, scalar.var_

(array([3.64016118e+04, 3.65589788e+04, 1.34217979e+04, ...,
        2.85106703e+01, 3.79238104e+00, 1.12015343e-01], shape=(3238,)),
 array([5.73178001e+08, 5.74492580e+08, 2.72592504e+07, ...,
        2.91915412e+00, 6.61509292e-02, 1.89172126e-03], shape=(3238,)))

In [19]:
X_scaled_df = pd.DataFrame(X_scaled, columns=X_train.columns)
X_test = pd.DataFrame(X_test, columns=X_train.columns)
X_scaled_df


,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,...,Feature_3229,Feature_3230,Feature_3231,Feature_3232,Feature_3233,Feature_3234,Feature_3235,Feature_3236,Feature_3237,Feature_3238
0,-0.756859,-0.756281,-0.768458,1.790937,-0.703377,-0.622357,0.757830,-0.722168,0.757830,0.038485,...,-1.903318,-1.903318,-0.795461,0.0,-0.386250,-1.156615,0.0,-0.208265,1.487482,-1.156615
1,-0.684660,-0.686690,-0.980408,0.279318,0.567037,0.393269,-0.768013,0.617284,-0.768013,-0.059407,...,-0.133632,-0.133632,-0.683039,0.0,-0.309647,-0.493571,0.0,-0.337385,0.542650,-0.493571
2,-0.381832,-0.380581,-0.235997,0.738313,-0.658969,-0.592319,0.689108,-0.672790,0.689108,-0.062952,...,-0.972867,-0.972867,-0.396858,0.0,-0.308825,-0.995803,0.0,-0.354606,1.137855,-0.995803
3,0.231255,0.232764,0.851293,0.408853,-1.186714,-0.927422,1.599928,-1.272312,1.599928,-0.063502,...,-0.486524,-0.486524,0.283390,0.0,-0.154057,-0.763600,0.0,-0.374634,0.852824,-0.763600
4,0.203521,0.203601,0.171085,-0.825213,0.093786,-0.031715,-0.308667,0.138226,-0.308667,-0.058745,...,0.680798,0.680798,0.173152,0.0,0.084921,0.703383,0.0,0.196797,-0.859371,0.703383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,0.433827,0.435696,0.886849,-0.217469,-0.847973,-0.717820,0.990873,-0.884254,0.990873,-0.060837,...,0.202267,0.202267,0.498488,0.0,-0.027807,-0.355315,0.0,0.162030,0.397758,-0.355315
311,-1.168752,-1.170949,-1.748648,1.717631,1.244825,1.018591,-1.401877,1.297774,-1.401877,-0.060507,...,-1.901386,-1.901386,-1.207844,0.0,-0.440121,-0.957524,0.0,0.652483,1.190021,-0.957524
312,0.035969,0.035381,0.101320,-0.395710,-0.196988,-0.259780,0.043164,-0.169541,0.043164,-0.058273,...,0.271247,0.271247,0.120722,0.0,-0.091668,-0.222419,0.0,-0.963045,0.088390,-0.222419
313,-0.472817,-0.471784,-0.331835,1.032608,-0.739118,-0.646287,0.814087,-0.762044,0.814087,-0.061786,...,-1.381913,-1.381913,-0.550115,0.0,-0.323534,-0.894995,0.0,0.392094,1.023034,-0.894995


In [20]:
feature_columns = X_train.columns

In [20]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=1e-5)
X_var = selector.fit_transform(X_scaled_df)
X_test = selector.transform(X_test)


print(f"Reduced from {X_scaled_df.shape[1]} to {X_var.shape[1]} features")

selected_mask = selector.get_support()

selected_features = [feature_columns[i] for i in range(len(selected_mask)) if selected_mask[i]]

Reduced from 3238 to 3123 features


In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(score_func=f_classif, k=100)  # Try k=100 to start

X_kbest = selector.fit_transform(X_scaled_df, y_train)
X_test = selector.transform(X_test)

# You can also get the selected column names if needed
selected_mask = selector.get_support()

selected_k_features = [feature_columns[i] for i in range(len(selected_mask)) if selected_mask[i]]

d:\classifier-algorithms\.venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [1719 1731 1733 1909 1910 1911 1912 1913 1914 1915 1916 1917 1918 1919
 1920 1921 1922 1923 1924 1925 1926 1927 1928 1929 1930 1931 1932 1933
 2109 2110 2111 2112 2113 2114 2115 2116 2117 2118 2119 2120 2121 2122
 2123 2124 2125 2126 2127 2128 2129 2130 2131 2132 2133 2248 2249 2255
 2259 2376 2377 2383 2387 2504 2505 2511 2515 2632 2633 2639 2640 2643
 2758 2759 2760 2761 2762 2763 2764 2765 2767 2769 2771 2772 2773 2886
 2887 2888 2889 2890 2891 2892 2893 2895 2897 2899 2900 2901 2937 2938
 2939 2940 2941 2977 2978 2979 2980 2981 3096 3097 3103 3106 3224 3225
 3231 3234] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
d:\classifier-algorithms\.venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

estimator = RandomForestClassifier(n_estimators=50, random_state=42)
feature_selector = RFE(estimator=estimator, n_features_to_select=100)
X_kbest = feature_selector.fit_transform(X_scaled_df, y_train)  # y = your target column (CLASS)
X_test = feature_selector.transform(X_test)

selected_mask = feature_selector.get_support()
selected_rfe_features = [feature_columns[i] for i in range(len(selected_mask)) if selected_mask[i]]

KeyboardInterrupt: 

In [24]:
X_test

array([[ 1.48108137,  1.31325536, -1.52203587, ..., -0.34722282,
        -0.88939853,  0.84372025],
       [ 0.57166613,  0.37893616, -0.80669617, ..., -0.18927466,
        -0.70589303,  1.82328967],
       [-0.6671397 , -0.59787158,  0.70165661, ..., -0.47183225,
        -0.09471213,  0.17241383],
       ...,
       [-0.83417749, -0.70886649,  0.96800111, ..., -0.1918633 ,
         0.34321402, -0.85403222],
       [-0.14947274, -0.22350309, -0.01692491, ..., -0.16057634,
        -0.27720937, -0.03273799],
       [-0.08795304, -0.17595864, -0.09316438, ...,  1.14720734,
         0.73039048, -0.47027103]], shape=(100, 919))

In [21]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier


estimator = RandomForestClassifier(n_estimators=50, random_state=42)
selector = SelectFromModel(estimator, prefit=False)

X_kbest = selector.fit_transform(X_scaled_df, y_train)  # y = your target column (CLASS)
X_test = selector.transform(X_test)


selected_mask = selector.get_support()
selected_rfe_features = [feature_columns[i] for i in range(len(selected_mask)) if selected_mask[i]]



In [24]:
X_kbest_df = pd.DataFrame(X_kbest, columns=selected_rfe_features)
X_test = pd.DataFrame(X_test, columns=selected_rfe_features)

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

models_config = {

    'Logistic Regression': {

        'model': LogisticRegression(random_state=42, max_iter=1000),
        'params': {
            'C': [0.001, 0.01, 0.1, 1, 10, 100],
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear', 'saga']
        }
    },

    'Random Forest': {

        'model': RandomForestClassifier(random_state=42, n_estimators=100),
        'params':{
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    },

    'SVM': {

        'model': SVC(random_state=42, probability=True),
        'params': {
            'C': [0.1, 1, 10, 100],
            'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
            'kernel': ['rbf', 'linear']
        }

    }

}

In [26]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

models = dict()

for name, config in models_config.items():
    print(f"Training {name}...")

    grid_search = GridSearchCV(estimator=config['model'], param_grid=config['params'], cv=cv, n_jobs=-1, scoring='accuracy', verbose=0)
    grid_search.fit(X_kbest_df, y_train)
    models[name] = {
        'model': grid_search.best_estimator_,
        'best_params': grid_search.best_params_,
        'cv_score': grid_search.best_score_
    }    
    print(f"Best CV Score: {grid_search.best_score_:.4f}")
    print(f"Best Parameters: {grid_search.best_params_}")

Training Logistic Regression...
Best CV Score: 0.6508
Best Parameters: {'C': 0.001, 'penalty': 'l2', 'solver': 'liblinear'}
Training Random Forest...
Best CV Score: 0.6444
Best Parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50}
Training SVM...
Best CV Score: 0.6222
Best Parameters: {'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}


In [27]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

results = {}

for name, model_info in models.items():
    model = model_info['model']

    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None

    accuracy = accuracy_score(y_test, y_pred)

    results[name] = {

        'accuracy': accuracy,
        'cv_score': model_info['cv_score'],
        'predictions': y_pred,
        'probabilities': y_pred_proba
    }

    print(f"\n{name}:")
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"CV Score: {model_info['cv_score']:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    



Logistic Regression:
Test Accuracy: 0.5900
CV Score: 0.6508

Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.60      0.63        58
           1       0.51      0.57      0.54        42

    accuracy                           0.59       100
   macro avg       0.59      0.59      0.58       100
weighted avg       0.60      0.59      0.59       100


Random Forest:
Test Accuracy: 0.6300
CV Score: 0.6444

Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.79      0.71        58
           1       0.59      0.40      0.48        42

    accuracy                           0.63       100
   macro avg       0.62      0.60      0.60       100
weighted avg       0.62      0.63      0.61       100


SVM:
Test Accuracy: 0.6300
CV Score: 0.6222

Classification Report:
              precision    recall  f1-score   support

           0       0.63      0.88      0.73        58
    

In [28]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(n_estimators=50, random_state=42, max_depth= 10, min_samples_leaf= 2, min_samples_split= 5)
clf_rf.fit(X_kbest_df, y_train)

y_pred_rf = clf_rf.predict(X_test)
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


[[48 10]
 [26 16]]
              precision    recall  f1-score   support

           0       0.65      0.83      0.73        58
           1       0.62      0.38      0.47        42

    accuracy                           0.64       100
   macro avg       0.63      0.60      0.60       100
weighted avg       0.63      0.64      0.62       100

